<h1 align="center">
    <img alt="Animi Motus AI Logo" src="images\logo.png" width="400" height="400">
    
    Den Loob - 01.01.1970
</h1>

<hr />

<div dir="rtl">
    <h2 align="center"><u>מטרת המחקר</u></h2>
    <div align="center">
    מטרת המחקר בשם Animi Motus* לעזור לאנשים עם הפרעות בזיהוי הבעות פנים. דוגמאות להפרעות כאלה הן 
    <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2737612/#b43">הפרעה דכאונית קשה</a>
    ו<a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2737612/#b32">סכיזופרניה</a>.
    </div>

\* משמעות של *"Animi Motus"* הינה *"רגשות"* בלטינית
</div>